# 1. SETTINGS

In [48]:
# libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import scipy.stats
from datetime import datetime

In [49]:
# import datasets
test  = pd.read_csv("../data/raw/Challenge_20180423.csv")
trade = pd.read_csv("../data/raw/Trade.csv")

In [50]:
# pandas options
pd.set_option("display.max_columns", None)
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# garbage collection
import gc
gc.enable()

# Merge Trade with Isin

In [8]:
tradedf = trade
#tradedf['TradeDateKey'] = pd.to_datetime(tradedf['TradeDateKey'], format='%Y%m%d', errors='ignore')

### Customer Features

In [100]:
def smooth_stat(dataset, grouping_features, type_of_stat, target_feature='CustomerInterest', alpha=10):
    '''dataset = set for the grouping
       grouping_feature = what we want to group on
       target_feature = what value should be calculated, here our target variable is by default,
       however, you can use other features, in the best case continous and highly correlated to the target value,
       it can help to prevent overfitting
       type_of_stat: mean min max std
              
       Note: for grouping variables where one has only one observation, 
       I decided to put zero instead f NA when we calculate standard deviation
       however, we can change it if needed
       '''
    #Make sure that grouping variables are categorical, otherwise as_index=False behaves wrongly, also calculating the std, mean etc.
    #dataset_cat = dataset.copy()
    #dataset_cat[grouping_features] = dataset_cat[grouping_features].apply(lambda x: x.astype('str'))
    
    K = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].count()
    
    if (type_of_stat=='std'):
        stat_feat_y = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].count()
        stat_feat_y[target_feature] = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].apply(lambda x : x.std()).as_matrix()
        stat_feat_y[target_feature][stat_feat_y[target_feature].isnull()] = 0
        global_stat_y = dataset_cat[target_feature].std()
        
    elif (type_of_stat=='min'):
        stat_feat_y = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].min()
        global_stat_y = dataset_cat[target_feature].min()
        
    elif (type_of_stat=='max'):
        stat_feat_y = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].max()
        global_stat_y = dataset_cat[target_feature].max()

    elif (type_of_stat=='mean'):
        stat_feat_y = dataset_cat.groupby(grouping_features, as_index=False)[target_feature].mean()
        global_stat_y = dataset_cat[target_feature].mean()
        
    stat_feat_y[target_feature] = (stat_feat_y[target_feature]*K[target_feature] + global_stat_y*alpha)/(K[target_feature]+alpha)
    
    return stat_feat_y

# Merging stats to the data and wrapper

In [105]:
def group_and_merge(data_for_calculation, data_to_merge, grouping_features='CustomerIdx'):
    for stat in ['min', 'max', 'std', 'mean']:
        temp = smooth_stat(data_for_calculation, grouping_features, type_of_stat=stat)          
        if len(grouping_features) > 1:
            pass
            '''temp_df = pd.DataFrame(columns=eval(f'{grouping_features}') + [f'{grouping_features}_{stat}'])
            temp_df[eval(f'{grouping_features}')] 
            temp_df[f'{grouping_features}_{stat}'] = temp.values
            data_to_merge = data_to_merge.merge(temp_df, left_on=eval(f'{grouping_features}'), right_on=f'{grouping_features}', how='left')'''
        else:
            temp_df = pd.DataFrame(columns=[f'{grouping_features}',f'{grouping_features}_{stat}'])
            temp_df[f'{grouping_features}'] = temp.index.values 
            temp_df[f'{grouping_features}_{stat}'] = temp
            data_to_merge = data_to_merge.merge(temp_df, on=f'{grouping_features}', how='left')
    return data_to_merge